In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [29]:
df_customers=pd.read_csv('/content/Customers.csv')
df_products=pd.read_csv('/content/Products.csv')
df_transactions=pd.read_csv('/content/Transactions.csv')
df=pd.merge(df_transactions,df_customers,on='CustomerID')
df=pd.merge(df,df_products,on='ProductID')
df.head(20)

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68
5,T00442,C0188,P067,2024-12-26 14:40:03,1,300.68,300.68,Anna Ball,South America,2022-05-17,ComfortLiving Bluetooth Speaker,Electronics,300.68
6,T00490,C0195,P067,2024-11-24 11:49:48,3,902.04,300.68,Jeremy Mclaughlin,South America,2024-09-17,ComfortLiving Bluetooth Speaker,Electronics,300.68
7,T00536,C0008,P067,2024-09-22 06:13:59,1,300.68,300.68,David Li,North America,2024-01-13,ComfortLiving Bluetooth Speaker,Electronics,300.68
8,T00564,C0157,P067,2024-12-07 17:57:40,3,902.04,300.68,Miguel Wong,North America,2024-01-30,ComfortLiving Bluetooth Speaker,Electronics,300.68
9,T00631,C0130,P067,2024-05-14 23:14:59,2,601.36,300.68,Robert Jones,South America,2023-04-19,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [33]:
def recommend_customers(df, user_id, num_recommendations=3):
    customer_features = df.groupby("CustomerID").agg(
        total_transactions=("TransactionID", "count"),
        average_transaction_value=("TotalValue", "mean"),
        total_quantity=("Quantity", "sum"),
        unique_products=("ProductID", "nunique")
    ).reset_index()

    product_category_counts = df.groupby(["CustomerID", "Category"])["ProductID"].count().unstack(fill_value=0)
    customer_features = customer_features.merge(product_category_counts, on="CustomerID")

    scaler = StandardScaler()
    numerical_cols = [col for col in customer_features.columns if col not in ["CustomerID"]]
    customer_features[numerical_cols] = scaler.fit_transform(customer_features[numerical_cols])

    customer_similarity = cosine_similarity(customer_features[numerical_cols])
    customer_similarity_df = pd.DataFrame(customer_similarity, index=customer_features["CustomerID"], columns=customer_features["CustomerID"])

    user_index = customer_features[customer_features["CustomerID"] == user_id].index[0]
    similar_customers = customer_similarity_df.iloc[user_index].sort_values(ascending=False)
    similar_customers = similar_customers[similar_customers.index != user_id]

    top_recommendations = similar_customers.head(num_recommendations).reset_index()
    top_recommendations.columns = ['CustomerID', 'SimilarityScore']
    return top_recommendations

user_id = input('Enter Customer ID: ')
recommendations = recommend_customers(df, user_id)
recommendations

Enter Customer ID: C0001


,CustomerID,SimilarityScore
0,C0069,0.940320
1,C0127,0.828037
2,C0035,0.793203


In [34]:
df = pd.merge(df_transactions, df_customers, on='CustomerID')
df = pd.merge(df, df_products, on='ProductID')

def recommend_customers(df, user_id, num_recommendations=3):
    customer_features = df.groupby("CustomerID").agg(
        total_transactions=("TransactionID", "count"),
        average_transaction_value=("TotalValue", "mean"),
        total_quantity=("Quantity", "sum"),
        unique_products=("ProductID", "nunique")
    ).reset_index()

    product_category_counts = df.groupby(["CustomerID", "Category"])["ProductID"].count().unstack(fill_value=0)
    customer_features = customer_features.merge(product_category_counts, on="CustomerID")

    scaler = StandardScaler()
    numerical_cols = [col for col in customer_features.columns if col not in ["CustomerID"]]
    customer_features[numerical_cols] = scaler.fit_transform(customer_features[numerical_cols])

    customer_similarity = cosine_similarity(customer_features[numerical_cols])
    customer_similarity_df = pd.DataFrame(customer_similarity, index=customer_features["CustomerID"], columns=customer_features["CustomerID"])

    user_index = customer_features[customer_features["CustomerID"] == user_id].index[0]
    similar_customers = customer_similarity_df.iloc[user_index].sort_values(ascending=False)
    similar_customers = similar_customers[similar_customers.index != user_id]

    top_recommendations = similar_customers.head(num_recommendations).reset_index()
    top_recommendations.columns = ['CustomerID', 'SimilarityScore']
    return top_recommendations

lookalike_data = {}

for i in range(20):
    user_id = df_customers['CustomerID'][i]
    recommendations = recommend_customers(df, user_id)
    lookalike_data[user_id] = []
    for index, row in recommendations.iterrows():
        lookalike_data[user_id].append([row['CustomerID'], row['SimilarityScore']])

lookalike_df = pd.DataFrame(list(lookalike_data.items()), columns=['CustomerID', 'Lookalikes'])
lookalike_df.to_csv('Lookalike.csv', index=False)